In [7]:
from azureml.core import Workspace, Environment, ComputeTarget, Experiment
from azureml.core.environment import CondaDependencies

ws = Workspace.from_config()

cpu_target = ComputeTarget(ws, name='tutorial-ma')

experiment = Experiment(ws, name='system-managed-sklearn-experiment')

# custom environment
env = Environment.get(ws, name='AzureML-sklearn-0.24-ubuntu18.04-py37-cpu')
myenv = env.clone('system_managed_sklean')
conda_dep = CondaDependencies()
conda_dep.add_conda_package('azureml-core')
myenv.python.conda_dependencies = conda_dep
_ = myenv.register(ws)

In [9]:
%%writefile system_environment_train.py

import joblib

from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
import sklearn

from azureml.core import Dataset
from azureml.core.run import Run
import numpy as np

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

X, y = load_diabetes(return_X_y=True)
model = Ridge().fit(X, y)

joblib.dump(model, 'sklearn_regression_model.pkl')

# Save datasets
np.savetxt('features.csv', X, delimiter=',')
np.savetxt('labels.csv', y, delimiter=',')

run = Run.get_context()
ws = run.experiment.workspace
datastore = ws.get_default_datastore()
datastore.upload_files(
    files=['./features.csv', './labels.csv'],
    target_path='sklearn_regression/',
    overwrite=True,
    )

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

# Register model
model = Model.register(
    workspace=ws,
    model_name='my-sklearn-model',
    model_path='./sklearn_regression_model.pkl',
    model_framework=Model.Framework.SCIKITLEARN,
    model_framework_version=sklearn.__version__,
    sample_input_dataset=input_dataset,
    sample_output_dataset=output_dataset,
    resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
    description='Ridge regression model to predictdisbetes progression.',
    tags={'area': 'diabetes', 'type': 'regression'},
)

print('Name:', model.name)
print('Version:', model.version)


Overwriting system_environment_train.py


In [10]:
from azureml.core import ScriptRunConfig

config = ScriptRunConfig(
    source_directory='.',
    script='system_environment_train.py',
    compute_target=cpu_target,
    environment=myenv,
)

run = experiment.submit(config)

aml_run = run.get_portal_url()
print(aml_run)


https://ml.azure.com/runs/system-managed-sklearn-experiment_1662797734_602425c3?wsid=/subscriptions/bbcac4cb-3713-4dc7-ad64-f8ffd50248c3/resourcegroups/rg-machinelearning-bigdatafecisa/workspaces/mlw-bigdatafecisa&tid=da060e56-5e46-475d-8b74-5fb187bd2177
